In [1]:
# Import Libraries
import pandas as pd

# Load Data
moneyball = pd.read_csv('mutant_moneyball.csv')

# Create dataframe
dataframe = pd.DataFrame(moneyball)

# Melt Data (Wide Format to Long Format)
dataframemelted = dataframe.melt(id_vars =['Member'], var_name = 'Company', value_name = 'Total Value ($)')



# Use extract to split first and last name into two rows
dataframemelted[['First','Last']] = dataframemelted['Member'].str.extract(r'([a-z]+)([A-Z][a-z]+)')

# Remove old member column
dataframemelted.drop(columns=['Member'], inplace=True)


# Utilize Split Twice, To split decade and Company name
dataframemelted[['Decade', 'Temperory']] = dataframemelted['Company'].str.split('_', expand=True)


dataframemelted.drop(columns=['Temperory'], inplace=True)


# Remove Unnecessary name classifications from variables
dataframemelted['Company'] = dataframemelted['Company'].str.replace('TotalValue', '', regex=False)

dataframemelted['Company'] = dataframemelted['Company'].str[4:]

dataframemelted['Decade'] = dataframemelted['Decade'].str.replace('TotalValue', '', regex=False)

# Had to ensure all the values were strings to delete $
dataframemelted['Total Value ($)'] = dataframemelted['Total Value ($)'].astype(str)

# Delete unnecessary $
dataframemelted['Total Value ($)'] = dataframemelted['Total Value ($)'].str.replace('$', '', regex=False)

# lower case variables for easier manipulation/search
dataframemelted['Last'] = dataframemelted['Last'].str.lower()

dataframemelted['Company'] = dataframemelted['Company'].str.lower()

# Reorder for logic
dataframemelted = dataframemelted[['First', 'Last', 'Company', 'Decade', 'Total Value ($)']]


# Empty in Total Value variables will be replaced with the mean of the company's sales
dataframemelted['Total Value ($)'] = pd.to_numeric(dataframemelted['Total Value ($)'], errors='coerce')

# Calculate the total earnings per character within each company across all decades
character_company_total = dataframemelted.groupby(['First', 'Last', 'Company'])['Total Value ($)'].transform('sum')

# Compute the per-decade average (divide total earnings by 4 decades)
character_company_avg = character_company_total / 4

# Replace NaN values with the per-character per-company average
dataframemelted['Total Value ($)'] = dataframemelted['Total Value ($)'].fillna(character_company_avg)

dataframemelted['Total Value ($)'] = dataframemelted['Total Value ($)'].round(2)

# Delete missing memembers
dataframemelted = dataframemelted.dropna(subset=['First'])
